1)	Se efectúa el registro en https://developer.twitter.com/en/apps 

2)	Creamos una aplicación.  

3)	Una vez completados los datos en los que se indica en qué se aplicarán los datos, la API proporciona una clave y un token para poder realizar la consulta.

4)	Instalación en anaconda/jupyter la librería a través del terminal (New/terminal)
   pip install tweepy

In [28]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import pandas as pd
import tweepy

# Autenticación en la API

In [29]:

import yaml
TWITTER_CONFIG_FILE = 'auth.yaml'

with open(TWITTER_CONFIG_FILE, 'r') as config_file:
    config = yaml.load(config_file)
    
print(type(config))

<class 'dict'>


In [26]:
#import json
#print(json.dumps(config, indent=4, sort_keys=True))

In [30]:
consumer_key = config['twitter']['consumer_key']
consumer_secret = config['twitter']['consumer_secret']
access_key = config['twitter']['access_key']
access_secret = config['twitter']['access_secret']

# Creamos la funcion que nos permite autenticarnos en la API

In [31]:
#Autenticación en la API de twitter con las credenciales de la aplicación

def auth(consumer_key, consumer_secret, access_key, access_secret):    
    auth = OAuthHandler(consumer_key, consumer_secret)    
    auth.set_access_token(access_key, access_secret)    
    return tweepy.API(auth) 

# DCP de los datos de twitter

In [32]:
#Búsqueda de tweets con el hastag indicado en q
#Se eliminan los retweets (-filter:retweets)
#Obtiene el texto completo (tweet_mode="extended")
#Devuelve un dataframe con la úntica columna que interesa para el análisis, el texto con el atributo (full_text)

def searchTwitter():
    api=auth(consumer_key, consumer_secret, access_key, access_secret)
    #search_result = api.search(search_text, lang="es",tweet_mode="extended", count=100)
    search =api.search(q="tan -filter:retweets", count=100, tweet_mode="extended", lang="es")
    outtweets = [[tweet.full_text] for tweet in search]
    tw_climatechange=pd.DataFrame(data=outtweets, columns=['tweet'])
    return tw_climatechange

In [33]:
#Se realiza las modificaciones sobre el dataframe con el objetivo de conseguir una estandarización para el análisis
#Supresión de cuentas a las que hace referencia (@XXXX) y aglutinación de múltiples espacios en uno
#Eliminación de páginas web (http,https y www) (str.replace('http\S+|www.\S+', '', case=False))
#Eliminación del símbolo # que hace referencia a los hastag
#Supresión de comentarios duplicados una vez que se elimina todo lo anterior

import re
def limpiezaDF(df):
    df['tweet']=df['tweet'].apply(lambda x: re.sub(r'@[A-Za-z0-9]+','', x)).apply(lambda x:' '.join(x.split()))
    df['tweet']=df['tweet'].str.replace('http\S+|www.\S+', '', case=False)
    df['tweet']=df['tweet'].str.replace('#', '', case=False) 
    #df['tweet']=df['tweet'].lower()
    df=df.drop_duplicates()
    return df

In [34]:
dfresult=searchTwitter()
dfresult=limpiezaDF(dfresult)
#Se decide almacenarlo en un archivo para poder reproducir el experimento con los mismo datos. 
dfresult.to_csv(r'C:\Users\Esperanza\Desktop\UOC\2año\TFM\tan.txt', header=True, index=None, sep=' ')
dfresult.head(2)

,tweet
0,Te voy a querer tan bonito que vas a sentir qu...
1,Me siento tan
